In [39]:
from collections import Counter
import os
import pandas as pd
import csv
import re
from gensim.models import Phrases, phrases

def getTexts(folder):
    fileToText = {}
    underscores = {}
    for root, dirs, files in os.walk(folder):
        for file in files:
                if '.txt' not in file: continue
                path = os.path.join(folder,file)
                f = open(path,'r',errors="ignore")
                text = f.read()
                name = file.split('.')[0]
                fileToText[name] = text
                f.close()
        return fileToText

def collectgrams(keylist, searchword, collectlist):
        for key in keylist:
                if searchword in key.split('_'):
                        collectlist.append(key) 

def getgrams(txt):
    infile = open(txt,'r')
    lines = infile.readlines()
    infile.close()
    allgrams = []
    for line in lines: 
        line = line.split(':')
        ngrams = line[1].strip()
        if '_' in ngrams: 
                ngrams = ngrams.strip().split(' ')
                for n in ngrams:
                        allgrams.append(n)
        else: continue
    return allgrams 

In [134]:
bigramdata = getTexts('/hpc/group/datap2023ecbc/Time Series Modeling Ranges/1636-1639_texts')
bigramtexts = list(bigramdata.values())
bigramnames = list(bigramdata.keys())
print(len(bigramnames))

756


In [135]:
training = []
for t in bigramtexts:
    words = t.split(' ')
    training.append(words)

print(len(training))
bigrammodel = Phrases(training, min_count=3, threshold= 0, scoring='npmi', connector_words=phrases.ENGLISH_CONNECTOR_WORDS, max_vocab_size =100000000)
print('bigram model trained')

trigrammodel = Phrases(bigrammodel[training], min_count=2, threshold= 0, scoring='npmi', connector_words=phrases.ENGLISH_CONNECTOR_WORDS)
print('trigram model trained')

total_bigram = bigrammodel.__dict__  # Get the model parameters as a dictionary
total_trigram = trigrammodel.__dict__


756
bigram model trained
trigram model trained


In [136]:
# combine bigram & trigram dictionary
if total_bigram and total_trigram:
    total_gram = {**total_bigram, **total_trigram}

# The output of the phrase model is a nested dictionary, 
# and all we need is the value of key=vocab
gram_dict = total_gram['vocab']
# with the line below to filter the grams by their frequency
filtered_dict = {key: value for key, value in gram_dict.items() if value > 1}
# the size of the grams in this folder
print(len(filtered_dict))

800813


In [137]:
#specify searchword：
searchwords = ['spanish']
searchword_keys = []

count = 0
with open('/hpc/group/datap2023ecbc/Ngrams_Spanish/Ngrams_1636_1639.txt', 'w') as file:
  
    # for key,value in filtered_dict.items():
    for searchword in searchwords: 
        collectgrams(filtered_dict.keys(), searchword, searchword_keys)
    print(len(searchword_keys))
    
# iterate the searchword_keys (a list), 
# each element in it is a N-gram word pair,
# write the N-gram with its freq(find it in the filtered_dict)
    for i,element in enumerate(searchword_keys):
        k = searchword_keys[i]
        freq = filtered_dict[k]
        file.write(f'{k}: {freq}\n')
            # print(k + ':'+ str(filtered_dict[k]))
        count +=1
        if count%500 == 0:
            print(count,"gram processed")
       

130


In [138]:
count = 0
outfile = open('/hpc/group/datap2023ecbc/Ngrams_Spanish/Ngrams_Spanish_1636_1639.txt', 'w')
for text in bigramtexts:
    #for outputting to txt file, specify here
    name = bigramnames[bigramtexts.index(text)]
    testtext = text.strip().split(' ')
    grams = []

    bgcount = Counter(b for b in bigrammodel[testtext] if len(b.split("_")) > 1)
    tgcount = Counter(t for t in trigrammodel[testtext] if len(t.split("_")) > 2)
    filtered_bgcount = {key: value for key, value in dict(bgcount).items() if value > 1}
    filtered_tgcount = {key: value for key, value in dict(tgcount).items() if value > 1}
    
    for searchword in searchwords: 
        collectgrams(filtered_bgcount.keys(), searchword, grams) 
        collectgrams(filtered_tgcount.keys(), searchword, grams)
    
    gramstring = ' '.join(grams)
    outfile.write(name+': '+gramstring+"\n")
    count += 1
    if count%50 == 0:
        print(count, "files processed")
outfile.close()
print('processing complete')
print(pd.DataFrame(dict(bgcount).values(), index=dict(bgcount).keys(), columns=['bigram frequency']).sort_values('bigram frequency', ascending=False).head(n=20))
print("\n")
print(pd.DataFrame(dict(tgcount).values(), index=dict(tgcount).keys(), columns=['trigram frequency']).sort_values('trigram frequency', ascending=False).head(n=20))


50 files processed
100 files processed
150 files processed
200 files processed
250 files processed
300 files processed
350 files processed
400 files processed
450 files processed
500 files processed
550 files processed
600 files processed
650 files processed
700 files processed
750 files processed
processing complete
                    bigram frequency
highness_quarter                   3
run_approach                       2
count_harrie                       2
count_harry                        2
count_william                      2
high_towards                       2
arable_land                        1
pleasure_take                      1
enemy_march                        1
several_quarter                    1
english_french                     1
lieutenant_colonel                 1
quarter_side                       1
count_solmes                       1
quarter_colonel                    1
keep_enemy                         1
french_english                     1
approach_right 